# Learning to Reweight Examples for Robust Deep Learning

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:

%cd /content/drive/MyDrive/ML_Project

/content/drive/MyDrive/ML_Project


In [3]:
%cd /content/drive/MyDrive/ML_Project/learning-to-reweight-examples/

/content/drive/MyDrive/ML_Project/learning-to-reweight-examples


In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from model import *
from data_loader import *
from wide_resnet import *
import matplotlib.pyplot as plt
from tqdm import tqdm
import IPython
import gc
import matplotlib
matplotlib.rcParams.update({'errorbar.capsize': 5})
import numpy as np
import random
import os
import csv

%matplotlib inline
%load_ext autoreload
%autoreload 2

In [5]:
hyperparameters = {
    'lr' : 0.1,
    'momentum' : 0.9,
    'batch_size' : 10,
    'num_iterations' : 100,
}

In [6]:
data_loader = get_cifar_loader(hyperparameters['batch_size'], n_val=1000, cifar_type="10", mode="train", run_type="unif_flip")
test_loader = get_cifar_loader(hyperparameters['batch_size'], n_val=1000, cifar_type="10", mode="test",run_type="unif_flip")

Files already downloaded and verified
50000
unif_flip
torch.Size([1000, 3, 32, 32])
torch.Size([1000])


/content/drive/MyDrive/ML_Project/learning-to-reweight-examples/data_loader.py:176: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:245.)
  self.data = torch.tensor(self.data).squeeze(0)


Files already downloaded and verified


In [7]:
def to_var(x, requires_grad=True):
    if torch.cuda.is_available():
        x = x.cuda()
    return Variable(x, requires_grad=requires_grad)

In [8]:
def build_model_wrn(cifar_type, model_type):
    net = Wide_ResNet(28, 10, 0.3, cifar_type)
    if(model_type == "baseline"):
      if os.path.isfile("cifar_unif_baseline.pt"):
        net.load_state_dict(torch.load('cifar_unif_baseline.pt'))
    elif(model_type == "reweight"):
      if os.path.isfile("cifar_unif_reweight.pt"):
        net.load_state_dict(torch.load('cifar_unif_reweight.pt'))
    if torch.cuda.is_available():
        net.cuda()
        torch.backends.cudnn.benchmark=True

    opt = torch.optim.SGD(net.params(),lr=hyperparameters["lr"])
    
    return net, opt

In [9]:
val_data = data_loader.dataset.data_val
val_labels = data_loader.dataset.labels_val

#val_data = val_data[:100]
#val_labels = val_labels[:100]

In [10]:
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

val_dataset = CIFAR_VAL(val_data, val_labels)
val_data_loader = DataLoader(dataset=val_dataset,
                             batch_size=20,
                             shuffle=True, num_workers=2)
    

In [11]:
from wide_resnet import Wide_ResNet

def train_lre_cifar_reweight():
    net, opt = build_model_wrn(cifar_type=10,model_type="reweight")
    global val_data
    global val_labels
    net_losses = []
    plot_step = 100

    smoothing_alpha = 0.9
    accuracy_log = []    
    lr_update = 1
    csv_net_losses = []
    csv_acc = []
    if os.path.isfile("acc_log_cifar_unif_flip.csv"):
      with open("acc_log_cifar_unif_flip.csv", "r") as csv_file:
        reader = csv.reader(csv_file, delimiter=',')
        for row in reader:
          csv_acc.append(list(np.float_(row)))
      csv_file.close()
      with open("loss_log_cifar_unif_flip.csv", "r") as csv_file:
        reader = csv.reader(csv_file, delimiter=',')
        for row in reader:
          csv_net_losses.append(list(np.float_(row)))
      csv_file.close()

    csv_net_losses = np.array(csv_net_losses)
    csv_acc = np.array(csv_acc)

    for i in tqdm(range(iter_start,iter_end)):
        if i > 40000:
          lr_update = 0.1
        if i > 50000:
          lr_update = 0.01
        opt.param_groups[0]['lr'] = lr_update * hyperparameters['lr']
        net.train()
        # Line 2 get batch of data
        image, labels = next(iter(data_loader))
        # since validation data is small I just fixed them instead of building an iterator
        # initialize a dummy network for the meta learning of the weights
        meta_net = Wide_ResNet(28, 10, 0.3, 10)
        meta_net.load_state_dict(net.state_dict())

        if torch.cuda.is_available():
            meta_net.cuda()

        image = to_var(image, requires_grad=False)
        labels = to_var(labels.type(torch.LongTensor), requires_grad=False)

        # Lines 4 - 5 initial forward pass to compute the initial weighted loss
        y_f_hat  = meta_net(image)
        cost = F.cross_entropy(y_f_hat,labels.type(torch.LongTensor).cuda(), reduce=False)
        eps = to_var(torch.zeros(cost.size()))
        l_f_meta = torch.sum(cost * eps)

        meta_net.zero_grad()
        
        # Line 6 perform a parameter update
        grads = torch.autograd.grad(l_f_meta, (meta_net.params()), create_graph=True)
        meta_net.update_params(lr_update*hyperparameters['lr'], source_params=grads)
        
        #select 300 images from the proposed 1000 validation images in the paper
        w_tilde_all = []
        for j in range(0,10):
          #print(j)
          image_val, labels_val = next(iter(val_data_loader))
          val_data_train = to_var(image_val, requires_grad=False)
          val_labels_train = to_var(labels_val, requires_grad=False)
          y_g_hat = meta_net(val_data_train)
          l_g_meta = F.cross_entropy(y_g_hat, val_labels_train)
          grad_eps = torch.autograd.grad(l_g_meta, eps, only_inputs=True, retain_graph=True)[0]

          # Line 11 computing and normalizing the weights
        
          w_tilde = torch.clamp(-grad_eps, min=0)
          #print(w_tilde.shape)
          w_tilde_all.append(w_tilde)

       # print(len(w_tilde_all))
        w_tilde_all = torch.vstack(w_tilde_all)
        #print(w_tilde_all.shape)
        w_tilde_mean = torch.mean(w_tilde_all, dim = 0)
        #print(w_tilde_mean.shape)
        norm_c = torch.sum(w_tilde_mean)

        if norm_c != 0:
            w = w_tilde_mean / norm_c
        else:
            w = w_tilde_mean

        # Lines 12 - 14 computing for the loss with the computed weights
        # and then perform a parameter update
        y_f_hat = net(image)
        cost = F.cross_entropy(y_f_hat, labels, reduce=False)
        l_f = torch.sum(cost * w)

        opt.zero_grad()
        l_f.backward()
        opt.step()

        net_losses.append(l_f.item())

        if i % plot_step == 0:
            net.eval()
            acc = []
            for itr,(test_img, test_label) in enumerate(test_loader):
                test_img = to_var(test_img, requires_grad=False)
                test_label = to_var(test_label, requires_grad=False)

                output = net(test_img)
                output = F.softmax(output, dim = 1)
                predicted = torch.argmax(output, dim = 1).int()
                acc.append((predicted.int() == test_label.int()).float())
            accuracy = torch.cat(acc,dim=0).mean()
            accuracy_log.append(np.array([i,accuracy.cpu()])[None])
            
            '''
            IPython.display.clear_output()
            fig, axes = plt.subplots(1, 2, figsize=(13,5))
            ax1, ax2 = axes.ravel()

            net_losses_plot = np.concatenate((csv_net_losses.flatten(),np.array(net_losses)), axis=0) if csv_net_losses.size else np.array(net_losses)
            ax1.plot(net_losses_plot, label='net_losses_reweighted')
            ax1.set_ylabel("Losses")
            ax1.set_xlabel("Iteration")
            ax1.legend()

            acc_log = np.concatenate((csv_acc,np.concatenate(accuracy_log,axis=0)), axis=0) if csv_acc.size else np.concatenate(accuracy_log,axis=0)
            ax2.plot(acc_log[:,0],100*acc_log[:,1],label='accuracy_reweighted')
            ax2.legend()
            ax2.set_ylabel('Accuracy %')
            ax2.set_xlabel('Iteration')
            plt.show()
            '''
    torch.save(net.state_dict(), 'cifar_unif_reweight.pt')        
        # return accuracy
    return np.concatenate(accuracy_log,axis=0), np.array(net_losses) 

In [34]:
iter_start = 10600
iter_end = 10800

In [ ]:
acc_log, net_losses = train_lre_cifar_reweight()
with open('acc_log_cifar_unif_flip.csv','a') as csvfile:
  np.savetxt(csvfile, acc_log, delimiter=',',fmt='%s', comments='')
  csvfile.close()
with open('loss_log_cifar_unif_flip.csv','a') as csvfile:
  np.savetxt(csvfile, net_losses, delimiter=',',fmt='%s', comments='')
  csvfile.close()

| Wide-Resnet 28x10


  0%|          | 0/200 [00:00<?, ?it/s]

| Wide-Resnet 28x10


  0%|          | 1/200 [00:35<1:58:42, 35.79s/it]

| Wide-Resnet 28x10


...

| Wide-Resnet 28x10


100%|█████████▉| 199/200 [28:36<00:08,  8.72s/it]

| Wide-Resnet 28x10


100%|██████████| 200/200 [28:45<00:00,  8.63s/it]
ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-35-01993bb38a3b>", line 1, in <cell line: 1>
    acc_log, net_losses = train_lre_cifar_reweight()
  File "<ipython-input-11-cadc5743e8c6>", line 135, in train_lre_cifar_reweight
    torch.save(net.state_dict(), 'cifar_unif_reweight.pt')
  File "/usr/local/lib/python3.10/dist-packages/torch/serialization.py", line 440, in save
    with _open_zipfile_writer(f) as opened_zipfile:
  File "/usr/local/lib/python3.10/dist-packages/torch/serialization.py", line 315, in _open_zipfile_writer
    return container(name_or_buffer)
  File "/usr/local/lib/python3.10/dist-packages/torch/serialization.py", line 288, in __init__
    super().__init__(torch._C.PyTorchFileWriter(str(name)))
RuntimeError: File cifar_unif_reweight.pt cannot be opened.

During handling of the above exception, an

In [32]:
iter_start = 10400
iter_end = 10600

In [33]:
acc_log, net_losses = train_lre_cifar_reweight()
with open('acc_log_cifar_unif_flip.csv','a') as csvfile:
  np.savetxt(csvfile, acc_log, delimiter=',',fmt='%s', comments='')
  csvfile.close()
with open('loss_log_cifar_unif_flip.csv','a') as csvfile:
  np.savetxt(csvfile, net_losses, delimiter=',',fmt='%s', comments='')
  csvfile.close()

| Wide-Resnet 28x10


  0%|          | 0/200 [00:00<?, ?it/s]

| Wide-Resnet 28x10


  0%|          | 1/200 [00:33<1:52:41, 33.98s/it]

| Wide-Resnet 28x10


...

 99%|█████████▉| 198/200 [26:34<00:15,  7.88s/it]

| Wide-Resnet 28x10


100%|█████████▉| 199/200 [26:43<00:08,  8.05s/it]

| Wide-Resnet 28x10


100%|██████████| 200/200 [26:50<00:00,  8.05s/it]
